In [1]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle

In [2]:
# face detection
# LOad the model
face_detection_model = './models/res10_300x300_ssd_iter_140000_fp16.caffemodel'
face_detection_proto = './models/deploy.prototxt.txt'
face_descriptor = './models/openface.nn4.small2.v1.t7'

detector_model = cv2.dnn.readNetFromCaffe(face_detection_proto,face_detection_model)
descriptor_model = cv2.dnn.readNetFromTorch(face_descriptor)

In [3]:
!dir = images

 Volume in drive C has no label.
 Volume Serial Number is 0434-922A

 Directory of C:\Users\hp\OneDrive\Desktop\FACE RECOGNITION\images

23-11-2023  11:04    <DIR>          .
23-11-2023  11:06    <DIR>          ..
23-11-2023  11:03    <DIR>          Aamir Khan
23-11-2023  11:03    <DIR>          Angelina Jolie
23-11-2023  11:03    <DIR>          Barack Obama
23-11-2023  11:03    <DIR>          Cristiano Ronaldo
23-11-2023  11:03    <DIR>          Donald Trump
23-11-2023  11:04    <DIR>          Elon Musk
23-11-2023  11:03    <DIR>          Joe Biden
23-11-2023  11:03    <DIR>          Leonardo DiCaprio
23-11-2023  11:03    <DIR>          Lionel Messi
23-11-2023  11:03    <DIR>          Robert Downey Jr
23-11-2023  11:03    <DIR>          Roger Federer
23-11-2023  11:03    <DIR>          Sachin Tendulkar
23-11-2023  11:03    <DIR>          Salman Khan
23-11-2023  11:03    <DIR>          Scarlett Johansson
23-11-2023  11:03    <DIR>          Tom Curise
               0 File(s)           

In [5]:
# consider sample image 
img = cv2.imread('./images/Sachin Tendulkar/2200.jpg')
cv2.imshow('Sample',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
def helper(image_path):
    img = cv2.imread(image_path)
# step -1 face detection
    image = img.copy()
    h,w = img.shape[:2]
    img_blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=False,crop = False)
    # set the input
    detector_model.setInput(img_blob)
    detections = detector_model.forward()
    
    if len(detections) > 0:
        i = np.argmax(detections[0,0,:,2])
        confidence = detections[0,0,i,2]
        if confidence > 0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startx,starty,endx,endy) = box.astype('int')
    
            roi = image[starty:endy,startx:endx].copy()
            faceblob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,0,0),swapRB=True,crop=True)
            descriptor_model.setInput(faceblob)
            vectors = descriptor_model.forward()

            return vectors
        return None
            # just for reference draw bounding
            # img_draw = image.copy()
            # cv2.rectangle(img_draw,(startx,starty),(endx,endy),(0,255,0))
    
    # cv2.imshow('sample',image)
    # cv2.imshow('draw_image',img_draw)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [16]:
helper('./images/Sachin Tendulkar/2200.jpg')

array([[-0.04844673,  0.0797378 , -0.09963042,  0.05219205,  0.02248179,
         0.20198311,  0.13852584, -0.06266055, -0.06219403,  0.01123158,
         0.00230623,  0.0771784 ,  0.01544621, -0.03280808, -0.01987841,
        -0.10874755, -0.09319472, -0.01268351,  0.10149096, -0.03454284,
         0.03998316, -0.1423884 , -0.11665358, -0.01645252,  0.07435046,
         0.00277319, -0.17686342, -0.14906754,  0.00644351,  0.10620247,
         0.08508851,  0.08288772, -0.08299837, -0.03749984,  0.04876524,
         0.05993138, -0.07223046, -0.06693253,  0.04339121, -0.0713728 ,
         0.14999197, -0.08112219,  0.04310589,  0.04121745, -0.09240643,
        -0.03493487,  0.11330164, -0.07407272, -0.12562124, -0.14079267,
        -0.07809556, -0.01990942,  0.07394505, -0.03580351,  0.08483167,
         0.13799942, -0.11584078,  0.19711757, -0.08613208, -0.01615907,
        -0.10057898,  0.07192262,  0.20695499, -0.21651323,  0.16872877,
         0.14802314,  0.04371177, -0.09877169, -0.1

In [21]:
data = dict(data=[],label=[])

In [22]:
folders = os.listdir('images')
# print(folders)
for folder in folders:
    filenames = os.listdir('images/{}'.format(folder))
    for filename in filenames:
      try:
            vector = helper('./images/{}/{}'.format(folder,filename))
            if vector is not None:
                data['data'].append(vector)
                data['label'].append(folder)
                print('Feature Extraction Successful')
      except:
          pass

Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Ex

In [23]:
data.keys()

dict_keys(['data', 'label'])

In [24]:
pd.Series(data['label']).value_counts()

Robert Downey Jr      125
Barack Obama          122
Donald Trump          122
Scarlett Johansson    119
Elon Musk             117
Lionel Messi          114
Roger Federer         111
Cristiano Ronaldo     109
Sachin Tendulkar      107
Salman Khan           107
Leonardo DiCaprio     106
Aamir Khan            103
Angelina Jolie         99
Joe Biden              98
Tom Curise             96
Name: count, dtype: int64

In [25]:
pickle.dump(data,open('data_face_features.pickle',mode='wb'))